In [1]:
# for some reason sometimes a few bytes are lost...
# so double-check that you got a correct signature before broadcasting the funding tx

from applets.core import get_connection, SecureAppletBase
from applets.util import secp256k1

class SingleUseKey(SecureAppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CD01", connection)

    def generate_fresh_key(self):
        return self.secure_request(b'\x20\x00')
        
    def get_pubkey(self):
        return self.secure_request(b'\x20\x01')

    def sign(self, msg:bytes):
        if len(msg)!=32:
            raise ValueError("Message hash should be 32 bytes long")
        return self.secure_request(b'\x20\x02'+msg)

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


In [4]:
app = SingleUseKey(connection)
app.select()
app.establish_secure_channel()
app.is_secure_channel_open

True

In [5]:
# generate a new random key
pub = app.generate_fresh_key()
pub.hex()

'037f61ff8ffa418c44886d37a6cf33e96087ace40bd2ab3232b30d3160a0e07b58'

In [6]:
len(pub)

33

In [7]:
# you can get this pubkey if you didn't sign using this key
app.get_pubkey() == pub

True

In [8]:
pub = app.get_pubkey()
print(len(pub), pub.hex())

33 037f61ff8ffa418c44886d37a6cf33e96087ace40bd2ab3232b30d3160a0e07b58


In [9]:
# sign message - only one message can be signed, then new key is generated
msg = b'5'*32
sig = app.sign(msg)
sig.hex()

'3045022100d984c017a76e62c43464378b49590d6b0e87e0b03e7d0f62bb5330adb67f71f602201dd2304bc2aea0da20b0754d2b7b8d22ed74e1e2c686b0470ed8c7620291e80b'

In [10]:
# verify it is valid:
sig_raw = secp256k1.ecdsa_signature_parse_der(sig)
pub_raw = secp256k1.ec_pubkey_parse(pub)
secp256k1.ecdsa_verify(sig_raw, msg, pub_raw)

True

In [11]:
connection.disconnect()